<a href="https://colab.research.google.com/github/aritri1/aritri1/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer= WordNetLemmatizer()
import keras
import pickle
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
!pip install tensorflow
!pip install optimizers
#import optimizers
import random 
from tensorflow.keras.optimizers import SGD
import io
import pandas as pd
from nltk import wordnet
import matplotlib.pyplot as plt
import string
import warnings 
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('popular', quiet=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


True

In [15]:
words=[]
classes=[]
documents=[]
patterns=[]
ignore_words=['?','!']
data_file=open('/content/intents.json').read()
intents=json.loads(data_file)
data_file

'{"intents": [\n        {"tag": "greeting",\n         "patterns": ["Hi there", "How are you", "Is anyone there?","Hey","Hola", "Hello", "Good day"],\n         "responses": ["Hello, thanks for asking", "Good to see you again", "Hi there, how can I help?"],\n         "context": [""]\n        },\n        {"tag": "goodbye",\n         "patterns": ["Bye", "See you later", "Goodbye", "Nice chatting to you, bye", "Till next time"],\n         "responses": ["See you!", "Have a nice day", "Bye! Come back again soon."],\n         "context": [""]\n        },\n        {"tag": "thanks",\n         "patterns": ["Thanks", "Thank you", "That\'s helpful", "Awesome, thanks", "Thanks for helping me"],\n         "responses": ["Happy to help!", "Any time!", "My pleasure"],\n         "context": [""]\n        },\n        {"tag": "noanswer",\n         "patterns": [],\n         "responses": ["Sorry, can\'t understand you", "Please give me more info", "Not sure I understand"],\n         "context": [""]\n        },

In [16]:
nltk.download('punkt')

for intent in intents ['intents']:
   for pattern in intent['patterns']:
      w=nltk.word_tokenize(pattern)
      words.extend(w)
      documents.append((w, intent['tag']))
      if intent['tag'] not in classes:
        classes.append(intent['tag'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
words=[lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

In [19]:
f=open('/content/chatbot_model.h5','r',errors = 'ignore')
raw=f.read()
raw = raw.lower()

In [20]:
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [21]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [26]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey", "how are you")
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me", "i am fine"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [27]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [28]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
hi
ROBO: I am glad! You are talking to me
how are you
ROBO: I am sorry! I don't understand you
bye
ROBO: Bye! take care..
